# Récuperer les données nécessaires à l'optimisation

On execute la préparation données et la partie ML

In [18]:
# les données
CO2reel = lasso_model.predict(datap)
dataprix = data2['Prix du véhicule']
dataprix = dataprix.iloc[list(datap.index)]
data_opti = pd.DataFrame.from_dict({'prix':dataprix, 'CO2reel':CO2reel})
data_opti.dropna(inplace=True)

N = 70

Ce dont on a besoin :

 1. Diviser les données en N catégories selon leur émissions de CO2

 2. Récuperer dans une dataframe un vecteur de taille N 
     - une colonne 'prix' contenant les prix moyens de chaque catégorie
     - une colonne 'CO2' contenant l'emission de CO2 au moyenne au sein de la catégorie

In [19]:

# Determine the min and max values for 'Co2_prf'
co2_min, co2_max = data_opti['CO2reel'].min(), data_opti['CO2reel'].max()

# Create bins with a width of 10 units
bin_width = int (( co2_max - co2_min)/ N ) 
bins = range(int(co2_min), int(co2_max) + bin_width, bin_width)


In [20]:
# Bin the 'Co2_prf' data
data_opti['CO2bins'] = pd.cut(data_opti['CO2reel'], bins, right=False)

# Group by the new 'Co2_prf_bins' column and count the number of vehicles in each bin
price_by_bin = data_opti.groupby('CO2bins')['prix'].mean()

# Get the mean CO2 from each bin
meanCO2_by_bin = data_opti.groupby('CO2bins')['CO2reel'].mean()

In [21]:
data_opti

,prix,CO2reel,CO2bins
0,34000.0,333.006825,"[330, 335)"
1,39700.0,164.457001,"[160, 165)"
3,34000.0,125.889710,"[125, 130)"
6,39300.0,98.195191,"[95, 100)"
7,34000.0,121.312736,"[120, 125)"
...,...,...,...
54157,24900.0,150.077620,"[150, 155)"
54158,22920.0,150.077620,"[150, 155)"
54202,32478.0,150.157768,"[150, 155)"
54203,32478.0,150.157768,"[150, 155)"


In [22]:
price_by_bin

CO2bins
[65, 70)      40195.255814
[70, 75)      43443.684524
[75, 80)      47369.054878
[80, 85)      43726.389027
[85, 90)      49612.720971
                  ...     
[395, 400)             NaN
[400, 405)             NaN
[405, 410)             NaN
[410, 415)             NaN
[415, 420)    28492.000000
Name: prix, Length: 71, dtype: float64

In [9]:
data_opti['CO2bins']

0        [330, 335)
1        [160, 165)
3        [125, 130)
6         [95, 100)
7        [120, 125)
            ...    
54157    [150, 155)
54158    [150, 155)
54202    [150, 155)
54203    [150, 155)
54204    [150, 155)
Name: CO2bins, Length: 39755, dtype: category
Categories (71, interval[int64, left]): [[65, 70) < [70, 75) < [75, 80) < [80, 85) ... [400, 405) < [405, 410) < [410, 415) < [415, 420)]

## Les variables finales qu'on va utiliser dans notre fonction minimize : p et CO2reel

Le vecteur prix

In [28]:
p= data_opti.groupby('CO2bins')['prix'].mean()
p.dropna(inplace=True)

In [29]:
p

CO2bins
[65, 70)      40195.255814
[70, 75)      43443.684524
[75, 80)      47369.054878
[80, 85)      43726.389027
[85, 90)      49612.720971
                  ...     
[365, 370)    32251.400000
[375, 380)    81490.000000
[380, 385)    26418.000000
[390, 395)    57350.000000
[415, 420)    28492.000000
Name: prix, Length: 61, dtype: float64

Le vecteur CO2reel

In [30]:
CO2reel = data_opti.groupby('CO2bins')['CO2reel'].mean()
CO2reel.dropna(inplace=True)

In [31]:
CO2reel

CO2bins
[65, 70)       66.925721
[70, 75)       73.159991
[75, 80)       77.357525
[80, 85)       83.142718
[85, 90)       87.484099
                 ...    
[365, 370)    368.559453
[375, 380)    378.791268
[380, 385)    383.031489
[390, 395)    393.231420
[415, 420)    417.048897
Name: CO2reel, Length: 61, dtype: float64